## Notebook für die Implementation der Enigma-Bombe

## Einleitung

Der folgende Code beschreibt das aufsetzen einer Enigma und das Knacken einer verschlüsselten Nachricht. Die Enigma ist eine Verschlüsselungsmaschine, die im zweiten Weltkrieg von den Deutschen verwendet wurde. Die Enigma-Bombe ist ein Gerät, das dazu dient, die Enigma zu knacken.

In [1]:
# import from the local file enigma.py
from sagEnigma import Enigma, Rotor, Reflector

### 0. Setup der Enigma 

In [13]:
rotor1 = Rotor([21, 25, 1, 17, 6, 8, 19, 24, 20, 15, 18, 3, 13, 7, 11, 23,0, 22, 12, 9, 16, 14, 5, 4, 2, 10], 26, 5, 2)
rotor2= Rotor([1, 3, 5, 7, 9, 11, 2, 15, 17, 19, 23, 21, 25, 13, 24, 4, 8, 22, 6, 0, 10, 12, 20, 18, 16, 14], 22, 24, 4)
rotor3 = Rotor([0, 9, 3, 10, 18, 8, 17, 20, 23, 1, 11, 7, 22, 19, 12, 2,16, 6, 25, 13, 15, 24, 5, 21, 14, 4], 5, 11, 24)
reflektor = Reflector([24, 17, 20, 7, 16, 18, 11, 3, 15, 23, 13, 6, 14, 10, 12, 8, 4, 1, 5, 25, 2, 22, 21, 9, 0, 19])

enigma_machine = Enigma([rotor1, rotor2, rotor3], reflektor)

message = "HELLO"
encrypted_message = enigma_machine.en_de_crypt(message)

encrypted_message

'GCIID'

### 1. Bombe

In [15]:
import threading
from queue import Queue

class Bombe:
    def __init__(self, max_wheel_val, max_position_val, max_reflector_val, crib, encrypted_word):
        self.max_wheel_val = max_wheel_val
        self.max_position_val = max_position_val
        self.max_reflector_val = max_reflector_val
        self.crib = crib
        self.encrypted_word = encrypted_word
        self.tasks = Queue()
        self.fill_tasks()

    def fill_tasks(self):
        for wheel1 in range(1, self.max_wheel_val):
            for wheel2 in range(1, self.max_wheel_val):
                for wheel3 in range(1, self.max_wheel_val):
                    for position1 in range(1, self.max_position_val):
                        for position2 in range(1, self.max_position_val):
                            for position3 in range(1, self.max_position_val):
                                for offset1 in range(1, self.max_position_val):
                                    for offset2 in range(1, self.max_position_val):
                                        for offset3 in range(1, self.max_position_val):
                                            for reflector in range(1, self.max_reflector_val):
                                                self.tasks.put(
                                                    [wheel1, wheel2, wheel3, position1, position2, position3,
                                                     offset1, offset2, offset3, reflector])

    def get_se_germans(self):
        try:
            while not self.tasks.empty():
                enigma_config = self.tasks.get()
                wheel_1, wheel_2, wheel_3, position_1, position_2, position_3, offset1, offset2, offset3, reflector = enigma_config
                threaded_enigma = Enigma.create([wheel_1, wheel_2, wheel_3], [position_1, position_2, position_3],
                                                [offset1, offset2, offset3], reflector)
                decrypted_text = threaded_enigma.en_de_crypt(self.encrypted_word)
                if decrypted_text == self.crib:
                    print(f"Decrypted Message: {decrypted_text}")
                    print(f"One possible setting of the Day: {enigma_config} go get those Fritzes!\n")
                self.tasks.task_done()
        except Exception as e:
            print(e)

    def crack(self):
        for _ in range(16):
            threading.Thread(target=self.get_se_germans).start()
        self.tasks.join()

In [16]:

bombe = Bombe(6, 26, 2, "HELLO", encrypted_message)